# Script notebook - criar banco de dados SQL para consultas eleitorais
---
### Ano 2024, UF RJ
### Tabelas
- perfil de eleitorado
- locais de votação
- resultados eleitorais

In [1]:
from tse import TseClient
from sql_utils import df_to_sql, listar_tabelas

- Inicializando cliente

In [2]:
client = TseClient()

- Menu de datasets disponíveis

In [3]:
for i, dataset in enumerate(client.datasets):
    print(f'{i} -> {dataset}')

0 -> candidatos-1934
1 -> candidatos-1986
2 -> resultados-2020-boletim-de-urna
3 -> resultados-2006
4 -> candidatos-1958
5 -> comparecimento-e-abstencao-2018
6 -> candidatos-1933
7 -> candidatos-1960
8 -> resultados-1998
9 -> candidatos-2014
10 -> pesquisas-eleitorais-2014
11 -> resultados-1933
12 -> comparecimento-e-abstencao-2022
13 -> resultados-2012
14 -> eleitorado-2006
15 -> pesquisas-eleitorais-atual
16 -> comparecimento-e-abstencao-2016
17 -> candidatos-2000
18 -> resultados-2022-correspondencias-esperadas-e-efetivadas-1-turno
19 -> candidatos-2004
20 -> mesarios-2024
21 -> pesquisas-eleitorais-2016
22 -> resultados-2014-correspondencias-esperadas-e-efetivadas-1-turno
23 -> eleitorado-2016
24 -> processual-2024
25 -> candidatos-1990
26 -> eleitorado-1998
27 -> eleitorado-2000
28 -> resultados-2016-correspondencias-esperadas-e-efetivadas-2-turno
29 -> resultados-1986
30 -> resultados-2018-correspondencias-esperadas-e-efetivadas-2-turno
31 -> resultados-1982
32 -> prestacao-de-co

## Escolhendo os datasets:
### eleitorado-2024
- perfil_eleitor_secao_2024_RJ: informacoes sobre perfil dos eleitores por secao no RJ
- eleitorado_local_votacao_2024: informacoes sobre locais de votacao

### resultados-2024
- votacao_secao_2024_RJ: informacoes sobre resultado das votacoes por secao no RJ


In [4]:
data = {} # armazena dataframes das tabelas selecionadas

- Dataset: perfil do eleitorado

In [5]:
perfil_eleitorado = client.find_dataset('eleitorado-2024', pattern='RJ')
perfil_eleitorado

{'https://cdn.tse.jus.br/estatistica/sead/odsele/perfil_eleitor_secao/perfil_eleitor_secao_2024_RJ.zip'}

- Dataset: locais de votacao (BRASIL)

In [6]:
locais_rj = client.find_dataset('eleitorado-2024', pattern='local')
locais_rj

{'https://cdn.tse.jus.br/estatistica/sead/odsele/eleitorado_locais_votacao/eleitorado_local_votacao_2024.zip'}

- Dataset: resultados da votação RJ

In [7]:
resultados = client.find_dataset('resultados-2024', pattern='RJ')
resultados

{'https://cdn.tse.jus.br/estatistica/sead/odsele/relatorio_resultado_totalizacao/Relatorio_Resultado_Totalizacao_2024_RJ.zip',
 'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2024_RJ.zip'}

- Download e carregamento de cada dataset em um dataframe

In [8]:
data.update(
    client.load_data_from_dirs(
        client.download_datasets(perfil_eleitorado)
    )
)

Files downloaded to data\perfil_eleitor_secao_2024_RJ


In [9]:
data.update(
    client.load_data_from_dirs(
        client.download_datasets(locais_rj)
    )
)

Files downloaded to data\eleitorado_local_votacao_2024


In [10]:
data.update(
    client.load_data_from_dirs(
        client.download_datasets(resultados)
    )
)

Files downloaded to data\Relatorio_Resultado_Totalizacao_2024_RJ
Files downloaded to data\votacao_secao_2024_RJ


In [11]:
data.keys()

dict_keys(['perfil_eleitor_secao_2024_RJ', 'eleitorado_local_votacao_2024', 'votacao_secao_2024_RJ'])

- Limpeza de algumas colunas

In [12]:
resultados_df = data.get('votacao_secao_2024_RJ')

In [13]:
resultados_df.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'CD_CARGO', 'DS_CARGO',
       'NR_VOTAVEL', 'NM_VOTAVEL', 'QT_VOTOS', 'NR_LOCAL_VOTACAO',
       'SQ_CANDIDATO', 'NM_LOCAL_VOTACAO', 'DS_LOCAL_VOTACAO_ENDERECO'],
      dtype='object')

In [14]:
resultados_columns = [
    'ANO_ELEICAO',
    'SG_UF',
    'CD_MUNICIPIO',
    'NR_ZONA', 
    'NR_LOCAL_VOTACAO',
    'NR_SECAO', 
    'NR_TURNO', 
    'CD_CARGO', 
    'DS_CARGO',
    'NR_VOTAVEL',
    'NM_VOTAVEL',
    'QT_VOTOS'
]
resultados_df = resultados_df[resultados_columns]

In [15]:
resultados_df.head()

,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,NR_SECAO,NR_TURNO,CD_CARGO,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS
0,2024,RJ,58130,96,1082,153,1,13,Vereador,44500,CAROLINE MIDORI DA COSTA SILVA,2
1,2024,RJ,58130,96,1856,333,1,13,Vereador,44500,CAROLINE MIDORI DA COSTA SILVA,5
2,2024,RJ,58696,157,1210,128,1,13,Vereador,30300,FERNANDO BRAZ DE LIMA,2
3,2024,RJ,58130,96,1953,179,1,13,Vereador,44500,CAROLINE MIDORI DA COSTA SILVA,5
4,2024,RJ,58696,157,1082,50,1,13,Vereador,30300,FERNANDO BRAZ DE LIMA,1


- Limpeza de algumas colunas

In [16]:
locais_df = data.get('eleitorado_local_votacao_2024')

In [17]:
locais_df.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'AA_ELEICAO', 'DT_ELEICAO', 'DS_ELEICAO',
       'NR_TURNO', 'SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'NR_ZONA',
       'NR_SECAO', 'CD_TIPO_SECAO_AGREGADA', 'DS_TIPO_SECAO_AGREGADA',
       'NR_SECAO_PRINCIPAL', 'NR_LOCAL_VOTACAO', 'NM_LOCAL_VOTACAO',
       'CD_TIPO_LOCAL', 'DS_TIPO_LOCAL', 'DS_ENDERECO', 'NM_BAIRRO', 'NR_CEP',
       'NR_TELEFONE_LOCAL', 'NR_LATITUDE', 'NR_LONGITUDE',
       'CD_SITU_LOCAL_VOTACAO', 'DS_SITU_LOCAL_VOTACAO', 'CD_SITU_ZONA',
       'DS_SITU_ZONA', 'CD_SITU_SECAO', 'DS_SITU_SECAO', 'CD_SITU_LOCALIDADE',
       'DS_SITU_LOCALIDADE', 'CD_SITU_SECAO_ACESSIBILIDADE',
       'DS_SITU_SECAO_ACESSIBILIDADE', 'QT_ELEITOR_SECAO',
       'QT_ELEITOR_ELEICAO_FEDERAL', 'QT_ELEITOR_ELEICAO_ESTADUAL',
       'QT_ELEITOR_ELEICAO_MUNICIPAL', 'NR_LOCAL_VOTACAO_ORIGINAL',
       'NM_LOCAL_VOTACAO_ORIGINAL', 'DS_ENDERECO_LOCVT_ORIGINAL'],
      dtype='object')

In [18]:
locais_columns = [
    'SG_UF',
    'CD_MUNICIPIO',
    'NR_ZONA', 
    'NR_LOCAL_VOTACAO',
    'NR_SECAO',
    'NM_MUNICIPIO',
    'NM_BAIRRO',
    'NR_LATITUDE',
    'NR_LONGITUDE',
    'QT_ELEITOR_SECAO'
]
locais_df = locais_df[locais_columns]

In [19]:
locais_df.head()

,SG_UF,CD_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,NR_SECAO,NM_MUNICIPIO,NM_BAIRRO,NR_LATITUDE,NR_LONGITUDE,QT_ELEITOR_SECAO
0,BA,39730,104,1147,62,LAPÃO,POVOADO DE CASAL I,-11.516413,-41.802394,262
1,BA,39730,104,1457,181,LAPÃO,CENTRO,-11.379024,-41.835959,307
2,SC,83798,36,1147,103,VIDEIRA,UNIVERSITÁRIO,-26.992234,-51.176671,388
3,SC,83798,36,1015,95,VIDEIRA,CENTRO,-27.002251,-51.157982,271
4,SP,72451,147,1074,178,VOTUPORANGA,VILA PAES,-20.414684,-49.971817,250


- Limpeza de algumas colunas

In [20]:
perfil_df = data.get('perfil_eleitor_secao_2024_RJ')

In [21]:
perfil_df.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO',
       'NM_LOCAL_VOTACAO', 'CD_GENERO', 'DS_GENERO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_FAIXA_ETARIA', 'DS_FAIXA_ETARIA',
       'CD_GRAU_ESCOLARIDADE', 'DS_GRAU_ESCOLARIDADE', 'CD_RACA_COR',
       'DS_RACA_COR', 'CD_IDENTIDADE_GENERO', 'DS_IDENTIDADE_GENERO',
       'CD_QUILOMBOLA', 'DS_QUILOMBOLA', 'CD_INTERPRETE_LIBRAS',
       'DS_INTERPRETE_LIBRAS', 'QT_ELEITORES_PERFIL', 'QT_ELEITORES_BIOMETRIA',
       'QT_ELEITORES_DEFICIENCIA', 'QT_ELEITORES_INC_NM_SOCIAL'],
      dtype='object')

In [22]:
perfil_columns = [
    'SG_UF',
    'CD_MUNICIPIO',
    'NR_ZONA', 
    'NR_LOCAL_VOTACAO',
    'NR_SECAO', 
    'DS_GENERO', 
    'DS_ESTADO_CIVIL',
    'DS_FAIXA_ETARIA',
    'DS_GRAU_ESCOLARIDADE',
    'DS_RACA_COR',
    'QT_ELEITORES_PERFIL'   
]
perfil_df = perfil_df[perfil_columns]

In [23]:
perfil_df.head()

,SG_UF,CD_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,NR_SECAO,DS_GENERO,DS_ESTADO_CIVIL,DS_FAIXA_ETARIA,DS_GRAU_ESCOLARIDADE,DS_RACA_COR,QT_ELEITORES_PERFIL
0,RJ,58009,255,1104,53,FEMININO,DIVORCIADO,35 a 39 anos,ENSINO MÉDIO INCOMPLETO,NÃO INFORMADO,1
1,RJ,60011,179,1449,312,FEMININO,CASADO,30 a 34 anos,SUPERIOR COMPLETO,Parda,1
2,RJ,60011,234,1163,44,FEMININO,SOLTEIRO,65 a 69 anos,ENSINO FUNDAMENTAL INCOMPLETO,NÃO INFORMADO,1
3,RJ,58696,156,1155,133,MASCULINO,SOLTEIRO,65 a 69 anos,ENSINO FUNDAMENTAL INCOMPLETO,NÃO INFORMADO,2
4,RJ,58491,110,1082,22,FEMININO,CASADO,65 a 69 anos,SUPERIOR COMPLETO,NÃO INFORMADO,1


- Finalmente: conversão para SQL database

In [24]:
df_to_sql(
    df=locais_df, 
    table_name='locais', 
    dbname='eleicoes_2024.sql')

df_to_sql(
    df=resultados_df,
    table_name='resultados',
    dbname='eleicoes_2024_rj.sql')

df_to_sql(
    df=perfil_df,
    table_name='eleitorado',
    dbname='eleicoes_2024_rj.sql')

Tabela "locais" foi criada no banco de dados "databases\eleicoes_2024.sql".
Tabela "resultados" foi criada no banco de dados "databases\eleicoes_2024_rj.sql".
Tabela "eleitorado" foi criada no banco de dados "databases\eleicoes_2024_rj.sql".


'eleitorado'

In [25]:
listar_tabelas('databases/eleicoes_2024_rj.sql')

Tabelas no banco de dados:
- resultados
- eleitorado


['resultados', 'eleitorado']

In [26]:
listar_tabelas('databases/eleicoes_2024.sql')

Tabelas no banco de dados:
- locais


['locais']

- Removendo arquivos temporários

In [27]:
TseClient.remove_data()

'Diretório data esvaziado'